In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import json, os, pickle, glob#, natsort
import corner
import astropy.units as u
import scipy.stats as sps

In [ ]:
import la_forge.diagnostics as dg
import la_forge.core as co
from targeted_cws_ng15 import post_processing as pp
from targeted_cws_ng15 import utils

In [ ]:
project_path = '/vast/palmer/home.grace/bbl29/targeted_cws_ng15'
outdir_path = '/vast/palmer/home.grace/bbl29/project/targeted_cws_ng15/data/chains'
save_loc = f'{project_path}/reports/figures/pub_figs'
dataset = 'ng15_v1p1'

## Log-uniform priors (detection runs)

In [ ]:
target_paths = glob.glob(f'{outdir_path}/{dataset}/*_det*')
[target_paths.pop(target_paths.index(tp)) for tp in np.flip(target_paths) if 'altskyloc' in tp]
[target_paths.pop(target_paths.index(tp)) for tp in np.flip(target_paths) if 'altskyloc' in tp]
[target_paths.pop(target_paths.index(tp)) for tp in np.flip(target_paths) if '49nHz' in tp]
target_strs = [tp.replace(f'{outdir_path}/{dataset}/','').replace('_det','') for tp in target_paths]
targets = np.copy(target_strs)
Nt = len(targets)
for i in range(Nt):
    targets[i] = targets[i].replace('_',' ')
    targets[i] = targets[i].replace('-','$-$')
    if '16nHz' in targets[i]:
        targets[i] = targets[i].replace(' 16nHz','')
    if '49nHz' in targets[i]:
        targets[i] = targets[i].replace(' 49nHz','')
    if 'varyfgw' in targets[i]:
        targets[i] = targets[i].replace(' varyfgw','')
targets = sorted(targets)
target_strs = sorted(target_strs)
target_paths = sorted(target_paths)
print(Nt,'targets')
print(targets)

In [ ]:
c = {}
for t, tp in zip(targets[len(c.keys()):], target_paths[len(c.keys()):]):
    print(t, tp)
    c[t] = co.Core(corepath=f'{tp}/core.h5', label=t)

In [ ]:
idx_start = 0
idx_end = -1
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    if t == 'NGC 3115':
        vary_fgw = True
    else:
        vary_fgw = False
    pp.corner_single(t_str, c[t], dataset, project_path, save_loc=save_loc, vary_crn=False,
                     vary_fgw=vary_fgw, detection=True, upper_limit=False, pub_fig=True,
                     target_fancy_name=None, truths=False, plot_mc_prior_mean=False)

### older code
    |
    |
    |
    |
    V

In [ ]:
idx_start = 0
idx_end = 3
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_uniform', save=True)

In [ ]:
idx_start = 3
idx_end = 6
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_uniform', save=True)

In [ ]:
idx_start = 6
idx_end = 9
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_uniform', save=True)

In [ ]:
idx_start = 9
idx_end = 12
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_uniform', save=True)

In [ ]:
idx_start = 12
idx_end = 15
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_uniform', save=True)

In [ ]:
idx_start = 15
idx_end = None
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_uniform', save=True)

In [ ]:
plt.close('all')

## Log-normal priors

In [ ]:
# Use the already compiled target names for this
target_paths = [f'{outdir_path}/{dataset}/{t}' for t in target_strs]

In [ ]:
c = {}
for t, tp in zip(targets[len(c.keys()):], target_paths[len(c.keys()):]):
    print(t)
    c[t] = co.Core(corepath=f'{tp}/core.h5', label=t)

In [ ]:
for t, tp in zip(targets, target_paths):
    prior_path = glob.glob(tp + '/*/priors.txt')[0]
    c[t].priors = np.loadtxt(prior_path, dtype=str, delimiter='\t')
    info_path = glob.glob(tp + '/*/runtime_info.txt')[0]
    c[t].runtime_info = np.loadtxt(info_path, dtype=str, delimiter='\t')

In [ ]:
# append param name
for t in targets:
    if 'log10_h0' in c[t].params:
        print('skipping', t, 'params')
    else:
        print('adding', t, 'params')
        c[t].params.append('log10_h0')

# append chain and prior
for t in targets:
    if len(c[t].params) == c[t].chain.shape[1]:
        print('skipping', t, 'chain')
    else:
        print('adding', t, 'chain')
        # log10_fgw
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_fgw' in c[t].runtime_info[i]][0]
        log10_fgw = float(line.replace('log10_fgw:Constant=',''))
        # log10_dL
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_dL' in c[t].runtime_info[i]][0]
        log10_dL = float(line.replace('log10_dL:Constant=',''))
        log10_dL_scaled = log10_dL + np.log10(u.Mpc.to(u.m)/c_light)
        # log10_mc chain
        log10_mc = c[t]('log10_mc',to_burn=False)
        log10_mc_scaled = log10_mc + np.log10(u.Msun.to(u.kg)*G/c_light**3)
        # calculate strain
        log10_h0 = 5*log10_mc_scaled/3 + 2*log10_fgw/3 - log10_dL_scaled + np.log10(2*np.pi)
        c[t].chain = np.vstack([c[t].chain.T,log10_h0]).T

# append prior - adjust for log-normal prior
for t in targets:
    if len(c[t].priors) == len(c[t].params) - 4:
        print('skipping', t, 'priors')
    else:
        print('adding', t, 'priors')
        # log10_fgw
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_fgw' in c[t].runtime_info[i]][0]
        log10_fgw = float(line.replace('log10_fgw:Constant=',''))
        # log10_dL
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_dL' in c[t].runtime_info[i]][0]
        log10_dL = float(line.replace('log10_dL:Constant=',''))
        log10_dL_scaled = log10_dL + np.log10(u.Mpc.to(u.m)/c_light)
        # get log10_mc prior
        pline = [p for p in c[t].priors if 'log10_mc' in p][0]
        log10_mc_mu = float(pline[pline.index('mu')+3:pline.index(', sigma')])
        log10_mc_sigma = float(pline[pline.index('sigma')+6:-1])
        #log10_mc_prior = np.array([pmin, pmax])
        log10_h0_mu = (5*(log10_mc_mu + np.log10(u.Msun.to(u.kg)*G/c_light**3))/3 +
                       2*log10_fgw/3 - log10_dL_scaled + np.log10(2*np.pi))
        r = ((np.max(c[t].chain[:,c[t].params.index('log10_h0')]) -
              np.min(c[t].chain[:,c[t].params.index('log10_h0')]))/
             (np.max(c[t].chain[:,c[t].params.index('log10_mc')]) -
              np.min(c[t].chain[:,c[t].params.index('log10_mc')])))
        log10_h0_sigma = r*log10_mc_sigma
        #log10_h0_prior_dist = sps.norm(loc=log10_h0_mu, scale=log10_h0_sigma)
        log10_h0_prior_str = f'log10_h0:Normal(mu={log10_h0_mu}, sigma={log10_h0_sigma})'
        c[t].priors = np.concatenate([c[t].priors, [log10_h0_prior_str]])

In [ ]:
idx_start = 14
idx_end = 15
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
idx_start = 0
idx_end = 3
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
idx_start = 3
idx_end = 6
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
idx_start = 6
idx_end = 9
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
idx_start = 9
idx_end = 12
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
idx_start = 12
idx_end = 15
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
idx_start = 15
idx_end = None
for t, t_str in zip(targets[idx_start:idx_end], target_strs[idx_start:idx_end]):
    print(t)
    fig = corner_single(t, t_str, c, 'log_normal', save=True)

In [ ]:
plt.close('all')

## HS 1630 12.5 year vs 15yr

In [ ]:
model_labels = ['12.5 yr', '15 yr']
datasets = ['ng12p5', 'ng15_v1p1']
target = 'HS_1630+2355'
target_paths = [f'{outdir_path}/{datasets[0]}/HS_1630+2355_det',
                f'{outdir_path}/{datasets[1]}/HS_1630+2355_det']

In [ ]:
c = {}
for ds, tp in zip(datasets, target_paths):
    print(ds)
    c[ds] = co.Core(corepath=f'{tp}/core.h5', label=ds, burn=0)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
for ds, color, ml in zip(datasets, ['C0','k'], model_labels):
    # get BF
    pline = [p for p in c[ds].priors if 'log10_mc' in p][0]
    pmin = float(pline[pline.index('pmin')+5:pline.index(',')])
    pmax = float(pline[pline.index('pmax')+5:pline.index(')')])
    BF = gorilla_bf(c[ds]('log10_mc'), min=pmin, max=pmax, nbins=nbins)
    ax.hist(c[ds]('log10_mc'), color=color, histtype='step', density=True, bins=20,
            range=[pmin,pmax], label=ml+f' (BF = {BF:1.2f})', lw=2)
x, y = pp.get_prior_distr(c[ds], 'log10_mc')
ax.plot(x, y, 'C2', rasterized=True, label='prior')
ax.legend(fontsize='medium')
ax.set_xlabel(r'$\log_{10}(\mathcal{M}_c/\rm{M}_\odot)$')
ax.set_ylabel('PDF')
ax.set_xlim([pmin,pmax])
ax.set_title('HS 1630+2355 (detection priors)')
fig.savefig(f'{save_loc}/HS_1630+2355_det_compare_datasets.pdf', format='pdf', dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
for ds, color, ml in zip(datasets, ['C0','k'], model_labels):
    # get ranges
    pline = [p for p in c[ds].priors if 'mc' in p and not 'log10' in p][0]
    pmin = float(pline[pline.index('pmin')+5:pline.index(',')])
    pmax = float(pline[pline.index('pmax')+5:pline.index(')')])
    # get UL
    ln_weights = utils.get_prior_log_weights(c[ds]('log10_mc'))
    mc_UL = utils.weighted_quantile(c[ds]('mc'), 0.95, sample_weight=np.exp(ln_weights))
    ax.hist(c[ds]('mc')/1e9, color=color, histtype='step', density=True, bins=20, weights=np.exp(ln_weights),
            label=ml+fr' ($\mathcal{{M}}_c$ UL = {mc_UL/1e9:2.1f} $\times10^9$ $\rm{{M}}_\odot$)', lw=2)
    ax.axvline(mc_UL/1e9, color=color)
#x, y = pp.get_prior_distr(c[ds], 'mc')
xlim = ax.get_xlim()
#ax.plot(x, y, 'C2', rasterized=True, label='prior')
ax.legend(fontsize='small')
ax.set_xlabel(r'$\mathcal{M}_c$ [$10^9$ $\rm{M}_\odot$]')
ax.set_ylabel('PDF')
ax.set_xlim([0,xlim[1]])
ax.set_title('HS 1630+2355 (upper limit priors)')
fig.savefig(f'{save_loc}/HS_1630+2355_UL_compare_datasets.pdf', format='pdf', dpi=600)
plt.show()

In [ ]:
dg.plot_chains([c['ng12p5'], c['ng15_v1p1']], pars=c['ng12p5'].params[:100], legend_labels=model_labels, ncols=4)

In [ ]:
dg.plot_chains([c['ng12p5'], c['ng15_v1p1']], pars=c['ng12p5'].params[100:], legend_labels=model_labels, ncols=4)

## NGC 3115 std vs 6 pulsar adv noise models

In [ ]:
from la_forge.rednoise import gorilla_bf

In [ ]:
model_labels = ['NGC 3115 @ 49 nHz w/ std noise', 'NGC 3115 @ 49 nHz w/ std + 6 psr custom noise']
datasets = ['ng15_v1p1', 'ng15_v1p1_custom6']
target = 'NGC_3115'
target_paths = [f'{outdir_path}/{datasets[0]}/NGC_3115_49nHz_det',
                f'{outdir_path}/{datasets[1]}/NGC_3115_49nHz_det_varycrn_fixedpoint']

In [ ]:
c = {}
for ds, tp in zip(datasets, target_paths):
    print(ds)
    c[ds] = co.Core(corepath=f'{tp}/core.h5', label=ds)

In [ ]:
save = True

BFs = {}
amp_param = 'log10_h0'
nbins = 6
pars_select = ['cos_inc', 'log10_h0', 'log10_mc']
#units = ['rad', 'rad', 'rad', 'calculated', r'$M_\odot$']
titles = [r'$\cos\iota$', r'$\log_{10}h_0$', r'$\log_{10}\mathcal{M}_c$']
labels = [r'$\cos\iota$', r'$\log_{10}h_0$', r'$\log_{10}(\mathcal{M}_c/\rm{M}_\odot)$']
quantiles = [0.16,0.5,0.84]

# plot
#labels = np.copy(titles)#[p+' \n ('+u+')' if u else p+' \n' for p,u in zip(titles, units)]
#labels[-1] = labels[-1] + r' [$M_\odot$]'
lines = []
ds = datasets[0]
idxs = [c[ds].params.index(p) for p in pars_select]
fig = corner.corner(c[ds].chain[c[ds].burn:, idxs], labels=labels,
                    plot_density=True, color='C0', plot_datapoints=False,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, rasterized=True, levels=(0.68, 0.95),
                    #titles=titles, show_titles=True, levels=(0.68, 0.95),
                    label_kwargs={'fontsize': 20}, title_kwargs={'fontsize': 14, 'color':'C0'})
# get BF
pline = [p for p in c[ds].priors if amp_param in p][0]
pmin = float(pline[pline.index('pmin')+5:pline.index(',')])
pmax = float(pline[pline.index('pmax')+5:pline.index(')')])
BFs[ds] = gorilla_bf(c[ds](amp_param), min=pmin, max=pmax, nbins=nbins)
lines.append(mlines.Line2D([],[],c='C0',label=model_labels[0]+
                           '\n'+f'BF = {np.round(BFs[ds],decimals=2)}'))

ds = datasets[1]
idxs = [c[ds].params.index(p) for p in pars_select]
fig = corner.corner(c[ds].chain[c[ds].burn:, idxs], fig=fig,#labels=labels,
                    plot_density=False, color='k', plot_datapoints=False,
                    no_fill_contours=True, rasterized=True,
                    titles=titles, show_titles=True,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, levels=(0.68, 0.95))
# get BF
pline = [p for p in c[ds].priors if amp_param in p][0]
pmin = float(pline[pline.index('pmin')+5:pline.index(',')])
pmax = float(pline[pline.index('pmax')+5:pline.index(')')])
BFs[ds] = gorilla_bf(c[ds](amp_param), min=pmin, max=pmax, nbins=nbins)
lines.append(mlines.Line2D([],[],c='k',label=model_labels[1]+
                           '\n'+f'BF = {np.round(BFs[ds],decimals=2)}'))
fig.legend(handles=lines, fontsize='medium')

# Extract the axes
axes = np.array(fig.axes).reshape((len(idxs), len(idxs)))

# Loop over the diagonal
for i, p in enumerate(pars_select):
    ax = axes[i, i]
    x, y = pp.get_prior_distr(c[ds], p)
    ax.plot(x, y, 'C2', rasterized=True)
    ax.set_xlim([x.min(),x.max()])
    if p == amp_param:
        bins = np.linspace(pmin, pmax, nbins+1)
        ax.axvspan(bins[0], bins[1], color='k', alpha=0.1)
# Loop over the histograms
for yi, p1 in enumerate(pars_select): # rows
    for xi, p2 in enumerate(pars_select[:yi]): # cols
        ax = axes[yi, xi]
        y, _ = pp.get_prior_distr(c[ds], p1)
        x, _ = pp.get_prior_distr(c[ds], p2)
        ax.set_xlim([x.min(),x.max()])
        ax.set_ylim([y.min(),y.max()])
if save:
    fig.savefig(f'{save_loc}/49nHz_noise_compare.pdf', format='pdf', dpi=600)

## NGC 3115 std vs 6 pulsar adv noise models at varied frequency

In [ ]:
model_labels = ['w/ std noise', 'w/ std + 6 pulsar custom noise']
datasets = ['ng15_v1p1', 'ng15_v1p1_custom6']
target = 'NGC_3115'
target_paths = [f'{outdir_path}/{datasets[0]}/NGC_3115_det_varyfgw',
                f'{outdir_path}/{datasets[1]}/NGC_3115_det_varyfgw_varycrn_fixedpoint']

In [ ]:
c = {}
for ds, tp in zip(datasets, target_paths):
    print(ds)
    c[ds] = co.Core(corepath=f'{tp}/core.h5', label=ds)

In [ ]:
save = True

pars_select = ['log10_fgw', 'cos_inc', 'log10_h0', 'log10_mc']
#units = ['rad', 'rad', 'rad', 'calculated', r'$M_\odot$']
titles = [r'$\log_{10}f_{\rm{GW}}$', r'$\cos\iota$', r'$\log_{10}h_0$',
          r'$\log_{10}\mathcal{M}_c$']
labels = [r'$\log_{10}f_{\rm{GW}}$ [Hz]', r'$\cos\iota$', r'$\log_{10}h_0$',
          r'$\log_{10}\mathcal{M}_c$ [$M_\odot$]']
quantiles = [0.16,0.5,0.84]

# plot
#labels = np.copy(titles)#[p+' \n ('+u+')' if u else p+' \n' for p,u in zip(titles, units)]
#labels[-1] = labels[-1] + r' [$M_\odot$]'
lines = []
ds = datasets[0]
idxs = [c[ds].params.index(p) for p in pars_select]
fig = corner.corner(c[ds].chain[c[ds].burn:, idxs], labels=labels,
                    plot_density=True, color='C0', plot_datapoints=False,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, rasterized=True, levels=(0.68, 0.95),
                    #titles=titles, show_titles=True, levels=(0.68, 0.95),
                    label_kwargs={'fontsize': 20}, title_kwargs={'fontsize': 14, 'color':'C0'})
lines.append(mlines.Line2D([],[],c='C0',label=model_labels[0]))
ds = datasets[1]
idxs = [c[ds].params.index(p) for p in pars_select]
fig = corner.corner(c[ds].chain[c[ds].burn:, idxs], fig=fig,#labels=labels,
                    plot_density=False, color='k', plot_datapoints=False,
                    no_fill_contours=True, rasterized=True,
                    titles=titles, show_titles=True,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, levels=(0.68, 0.95))
lines.append(mlines.Line2D([],[],c='k',label=model_labels[1]))
fig.legend(handles=lines, fontsize='xx-large')

# Extract the axes
axes = np.array(fig.axes).reshape((len(idxs), len(idxs)))

# Loop over the diagonal
for i, p in enumerate(pars_select):
    ax = axes[i, i]
    x, y = pp.get_prior_distr(c[ds], p)
    ax.plot(x, y, 'C2', rasterized=True)
    ax.set_xlim([x.min(),x.max()])
# Loop over the histograms
for yi, p1 in enumerate(pars_select): # rows
    for xi, p2 in enumerate(pars_select[:yi]): # cols
        ax = axes[yi, xi]
        y, _ = pp.get_prior_distr(c[ds], p1)
        x, _ = pp.get_prior_distr(c[ds], p2)
        ax.set_xlim([x.min(),x.max()])
        ax.set_ylim([y.min(),y.max()])
if save:
    fig.savefig(f'{save_loc}/noise_compare.pdf', format='pdf', dpi=600)

## NGC 3115 std vs pulsars removed

In [ ]:
model_labels = ['Full PTA', 'PSR J1024-0719 removed', 'PSR J1012+5307 removed', 'PSR J1713+0747 removed']
datasets = ['ng15_v1p1', 'ng15_v1p1_dropJ1024', 'ng15_v1p1_dropJ1012', 'ng15_v1p1_dropJ1713']
target = 'NGC_3115'
target_paths = [f'{outdir_path}/{dataset}/NGC_3115_det_varyfgw' for dataset in datasets]

In [ ]:
c = {}
for ds, tp in zip(datasets, target_paths):
    print(ds)
    c[ds] = co.Core(corepath=f'{tp}/core.h5', label=ds)

In [ ]:
save = True

pars_select = ['log10_fgw', 'cos_inc', 'log10_h0', 'log10_mc']
#units = ['rad', 'rad', 'rad', 'calculated', r'$M_\odot$']
titles = [r'$\log_{10}f_{\rm{GW}}$', r'$\cos\iota$', r'$\log_{10}h_0$',
          r'$\log_{10}\mathcal{M}_c$']
labels = [r'$\log_{10}f_{\rm{GW}}$ [Hz]', r'$\cos\iota$', r'$\log_{10}h_0$',
          r'$\log_{10}\mathcal{M}_c$ [$M_\odot$]']
quantiles = [0.16,0.5,0.84]

# plot
#labels = np.copy(titles)#[p+' \n ('+u+')' if u else p+' \n' for p,u in zip(titles, units)]
#labels[-1] = labels[-1] + r' [$M_\odot$]'
lines = []
ds = datasets[0]
BF = np.round(pp.get_bayes_fac(c[ds]), decimals=2)
idxs = [c[ds].params.index(p) for p in pars_select]
fig = corner.corner(c[ds].chain[c[ds].burn:, idxs], labels=labels,
                    plot_density=True, color='k', plot_datapoints=False,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, rasterized=True, levels=(0.68, 0.95),
                    #titles=titles, show_titles=True, levels=(0.68, 0.95),
                    label_kwargs={'fontsize': 20}, title_kwargs={'fontsize': 14, 'color':'C0'})
lines.append(mlines.Line2D([],[],c='k',label=f'{model_labels[0]} | BF = {BF}'))
for i in range(1,len(datasets)):
    ds = datasets[i]
    color = f'C{i-1}'
    BF = np.round(pp.get_bayes_fac(c[ds]), decimals=2)
    idxs = [c[ds].params.index(p) for p in pars_select]
    fig = corner.corner(c[ds].chain[c[ds].burn:, idxs], fig=fig,#labels=labels,
                        plot_density=False, color=color, plot_datapoints=False,
                        no_fill_contours=True, rasterized=True,
                        titles=titles, show_titles=True,
                        #quantiles=quantiles, title_quantiles=quantiles,
                        hist_kwargs={'density':True}, levels=(0.68, 0.95))
    lines.append(mlines.Line2D([],[],c=color,label=f'{model_labels[i]} | BF = {BF}'))
fig.legend(handles=lines, fontsize='xx-large')

# Extract the axes
axes = np.array(fig.axes).reshape((len(idxs), len(idxs)))

# Loop over the diagonal
for i, p in enumerate(pars_select):
    ax = axes[i, i]
    x, y = pp.get_prior_distr(c[ds], p)
    ax.plot(x, y, 'k', alpha=0.5, lw=1, linestyle='dashed', rasterized=True)
    ax.set_xlim([x.min(),x.max()])
# Loop over the histograms
for yi, p1 in enumerate(pars_select): # rows
    for xi, p2 in enumerate(pars_select[:yi]): # cols
        ax = axes[yi, xi]
        y, _ = pp.get_prior_distr(c[ds], p1)
        x, _ = pp.get_prior_distr(c[ds], p2)
        ax.set_xlim([x.min(),x.max()])
        ax.set_ylim([y.min(),y.max()])
if save:
    fig.savefig(f'{save_loc}/datasets_compare.pdf', format='pdf', dpi=600)

## HS 1630, Real vs alt sky location

In [ ]:
targets = ['Real sky location', 'Alt sky location']
t_str = 'HS_1630+2355'
target_paths = glob.glob(f'{outdir_path}/{dataset}/{t_str}*UL')
target_paths

In [ ]:
c = {}
for t, tp in zip(targets[len(c.keys()):], target_paths[len(c.keys()):]):
    print(t)
    c[t] = co.Core(corepath=f'{tp}/core.h5', label=t)

In [ ]:
for t, tp in zip(targets, target_paths):
    prior_path = glob.glob(tp + '/*/priors.txt')[0]
    c[t].priors = np.loadtxt(prior_path, dtype=str, delimiter='\t')
    info_path = glob.glob(tp + '/*/runtime_info.txt')[0]
    c[t].runtime_info = np.loadtxt(info_path, dtype=str, delimiter='\t')

In [ ]:
# append param name
for t in targets:
    if 'log10_h0' in c[t].params:
        print('skipping', t, 'params')
    else:
        print('adding', t, 'params')
        c[t].params.append('log10_h0')

# append chain and prior
for t in targets:
    if len(c[t].params) == c[t].chain.shape[1]:
        print('skipping', t, 'chain')
    else:
        print('adding', t, 'chain')
        # log10_fgw
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_fgw' in c[t].runtime_info[i]][0]
        log10_fgw = float(line.replace('log10_fgw:Constant=',''))
        # log10_dL
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_dL' in c[t].runtime_info[i]][0]
        log10_dL = float(line.replace('log10_dL:Constant=',''))
        log10_dL_scaled = log10_dL + np.log10(u.Mpc.to(u.m)/c_light)
        # log10_mc chain
        log10_mc = c[t]('log10_mc',to_burn=False)
        log10_mc_scaled = log10_mc + np.log10(u.Msun.to(u.kg)*G/c_light**3)
        # calculate strain
        log10_h0 = 5*log10_mc_scaled/3 + 2*log10_fgw/3 - log10_dL_scaled + np.log10(2*np.pi)
        c[t].chain = np.vstack([c[t].chain.T,log10_h0]).T

# append prior
for t in targets:
    if len(c[t].priors) == len(c[t].params) - 4:
        print('skipping', t, 'priors')
    else:
        print('adding', t, 'priors')
        # log10_fgw
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_fgw' in c[t].runtime_info[i]][0]
        log10_fgw = float(line.replace('log10_fgw:Constant=',''))
        # log10_dL
        line = [c[t].runtime_info[i] for i in range(len(c[t].runtime_info))
                if 'log10_dL' in c[t].runtime_info[i]][0]
        log10_dL = float(line.replace('log10_dL:Constant=',''))
        log10_dL_scaled = log10_dL + np.log10(u.Mpc.to(u.m)/c_light)
        # get log10_mc prior
        pline = [p for p in c[t].priors if 'log10_mc' in p][0]
        pmin = float(pline[pline.index('pmin')+5:pline.index(', pmax')])
        pmax = float(pline[pline.index('pmax')+5:-1])
        log10_mc_prior = np.array([pmin, pmax])
        log10_mc_prior_scaled = log10_mc_prior + np.log10(u.Msun.to(u.kg)*G/c_light**3)
        # calculate strain prior
        log10_h0_prior = 5*log10_mc_prior_scaled/3 + 2*log10_fgw/3 - log10_dL_scaled + np.log10(2*np.pi)
        log10_h0_prior_str = f'log10_h0:Uniform(pmin={log10_h0_prior[0]}, pmax={log10_h0_prior[1]})'
        c[t].priors = np.concatenate([c[t].priors, [log10_h0_prior_str]])

In [ ]:
save = True

pars_select = ['cos_inc','phase0','psi', 'log10_h0', 'log10_mc']
#units = ['rad', 'rad', 'rad', 'calculated', r'$M_\odot$']
titles = [r'$\cos\iota$', '$\Phi_0$', r'$\psi$', r'$\log_{10}h_0$', r'$\log_{10}\mathcal{M}_c$']
labels = [r'$\cos\iota$', '$\Phi_0$', r'$\psi$', r'$\log_{10}h_0$', r'$\log_{10}\mathcal{M}_c$ [$M_\odot$]']
quantiles = [0.16,0.5,0.84]

# plot
#labels = np.copy(titles)#[p+' \n ('+u+')' if u else p+' \n' for p,u in zip(titles, units)]
#labels[-1] = labels[-1] + r' [$M_\odot$]'
lines = []
t = targets[0]
idxs = [c[t].params.index(p) for p in pars_select]
fig = corner.corner(c[t].chain[c[t].burn:, idxs], labels=labels,
                    plot_density=True, color='C0', plot_datapoints=False,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True},
                    titles=titles, show_titles=True, levels=(0.68, 0.95),
                    label_kwargs={'fontsize': 20}, title_kwargs={'fontsize': 14})
lines.append(mlines.Line2D([],[],c='C0',label=t))
t = targets[1]
idxs = [c[t].params.index(p) for p in pars_select]
fig = corner.corner(c[t].chain[c[t].burn:, idxs], fig=fig,#labels=labels,
                    plot_density=False, color='k', plot_datapoints=False,
                    no_fill_contours=True,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, levels=(0.68, 0.95))
lines.append(mlines.Line2D([],[],c='k',label=t))
fig.legend(handles=lines, fontsize='xx-large')

# Extract the axes
axes = np.array(fig.axes).reshape((len(idxs), len(idxs)))

# Loop over the diagonal
for i, p in enumerate(pars_select):
    ax = axes[i, i]
    x, y = get_prior_distr(c[t], p)
    ax.plot(x, y, 'C2')
    ax.set_xlim([x.min(),x.max()])
    if p == 'log10_mc':
        ax.axvline(get_mc_prior_mean(t_str), color='C1')
        #ax.axvline(get_mc_prior_mean(t_str)+0.3, color='r', alpha=0.5, linestyle='dashed')
        #ax.axvline(get_mc_prior_mean(t_str)-0.3, color='r', alpha=0.5, linestyle='dashed')
        ylim = ax.get_ylim()
        ax.fill_between([get_mc_prior_mean(t_str)-0.3,get_mc_prior_mean(t_str)+0.3],
                        [ylim[0],ylim[0]],[ylim[1],ylim[1]], color='C1', alpha=0.2)
# Loop over the histograms
for yi, p1 in enumerate(pars_select): # rows
    for xi, p2 in enumerate(pars_select[:yi]): # cols
        ax = axes[yi, xi]
        y, _ = get_prior_distr(c[t], p1)
        x, _ = get_prior_distr(c[t], p2)
        ax.set_xlim([x.min(),x.max()])
        ax.set_ylim([y.min(),y.max()])
if save:
    fig.savefig(f'{save_loc}/HS1630_altskyloc.pdf', format='pdf', dpi=600)

In [ ]:
save = True

pars_select = ['cos_inc','phase0','psi', 'log10_h0', 'log10_mc']
#units = ['rad', 'rad', 'rad', 'calculated', r'$M_\odot$']
titles = [r'$\cos\iota$', '$\Phi_0$', r'$\psi$', r'$\log_{10}h_0$', r'$\log_{10}\mathcal{M}_c$']
labels = [r'$\cos\iota$', '$\Phi_0$', r'$\psi$', r'$\log_{10}h_0$', r'$\log_{10}\mathcal{M}_c$ [$M_\odot$]']
quantiles = [0.16,0.5,0.84]

# plot
#labels = np.copy(titles)#[p+' \n ('+u+')' if u else p+' \n' for p,u in zip(titles, units)]
#labels[-1] = labels[-1] + r' [$M_\odot$]'
lines = []
t = targets[0]
idxs = [c[t].params.index(p) for p in pars_select]
fig = corner.corner(c[t].chain[c[t].burn:, idxs], labels=labels,
                    plot_density=True, color='C0', plot_datapoints=False,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, rasterized=True,
                    titles=titles, show_titles=True, levels=(0.68, 0.95),
                    label_kwargs={'fontsize': 20}, title_kwargs={'fontsize': 14, 'color':'C0'})
lines.append(mlines.Line2D([],[],c='C0',label=t))
t = targets[1]
idxs = [c[t].params.index(p) for p in pars_select]
fig = corner.corner(c[t].chain[c[t].burn:, idxs], fig=fig,#labels=labels,
                    plot_density=False, color='k', plot_datapoints=False,
                    no_fill_contours=True, rasterized=True,
                    #quantiles=quantiles, title_quantiles=quantiles,
                    hist_kwargs={'density':True}, levels=(0.68, 0.95))
lines.append(mlines.Line2D([],[],c='k',label=t))
fig.legend(handles=lines, fontsize='xx-large')

fig.suptitle(f'HS 1630+2355 comparing sky locs, \n'
             f'Real: RA = {RA[0]}, DEC = {DEC[0]} \n'
             f'Alt: RA = {RA[1]}, DEC = {DEC[1]} \n'
             r'(fixed CRN, frequency, $d_L$)' + f' \n'
             f'log-uniform mass priors \n',
             fontsize=16, x=1, y=0.9, horizontalalignment='right')

# Extract the axes
axes = np.array(fig.axes).reshape((len(idxs), len(idxs)))

# Loop over the diagonal
for i, p in enumerate(pars_select):
    ax = axes[i, i]
    x, y = get_prior_distr(c[t], p)
    ax.plot(x, y, 'C2', rasterized=True)
    ax.set_xlim([x.min(),x.max()])
    if p == 'log10_mc':
        ax.axvline(get_mc_prior_mean(t_str), color='C1', rasterized=True)
        #ax.axvline(get_mc_prior_mean(t_str)+0.3, color='r', alpha=0.5, linestyle='dashed')
        #ax.axvline(get_mc_prior_mean(t_str)-0.3, color='r', alpha=0.5, linestyle='dashed')
        ylim = ax.get_ylim()
        ax.fill_between([get_mc_prior_mean(t_str)-0.3,get_mc_prior_mean(t_str)+0.3],
                        [ylim[0],ylim[0]],[ylim[1],ylim[1]], color='C1', alpha=0.2, rasterized=True)
# Loop over the histograms
for yi, p1 in enumerate(pars_select): # rows
    for xi, p2 in enumerate(pars_select[:yi]): # cols
        ax = axes[yi, xi]
        y, _ = get_prior_distr(c[t], p1)
        x, _ = get_prior_distr(c[t], p2)
        ax.set_xlim([x.min(),x.max()])
        ax.set_ylim([y.min(),y.max()])